## Dependencies

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

## Prep csv

In [5]:
import csv
import os
import numpy as np
import time

In [4]:
#num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords = 501

In [5]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [6]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [3]:
class_names = ['Confidence', 'Engagement', 'Positive Attitude', 'Active interest', 'Body Language']

In [6]:
class_index = 0
window_duration = 20  # Duration for each class window in seconds
cap = cv2.VideoCapture(0)
start_time = time.time()  # Start time for the current class window

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()
        class_name = class_names[class_index]

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw class name on the image
        cv2.putText(image, f"Class: {class_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw landmarks
        # ...

        # Check if the class window duration has elapsed
        elapsed_time = time.time() - start_time
        if elapsed_time >= window_duration:
            # Reset timer and increment class index
            start_time = time.time()
            class_index += 1
            if class_index >= len(class_names):
                break

        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Concatenate rows
            row = pose_row + face_row

            # Append class name
            row.insert(0, class_name)

            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


## share in teams https://sihirio.sharepoint.com/:f:/s/ML934/Egm-1iz4-XNEmYum8sAKfnYBzU3ugXgy9iMWdKboWaEDAg?e=JtPKk2